# Elections Ontario - Technical Test
#### This notebook demonstrates:
#### - loading (ingestion), processing, and automated processing of the provided census data
#### - creating requested metrics
#### - populating the provided template with the requested metrics
#### - showcasing innovation by leveraging Streamlit for data visualization
#### - sharing recommendations on how to improve the process

# Load Data

In [5]:
# Load libraries
import pandas as pd
# import re
from openpyxl import load_workbook
from copy import copy

In [6]:
# Define variables
pop_summary_file = 'SRDA EO-2025-30 Technical Test Population summary.xlsx'
census_file = 'SRDA EO-2025039 Technical Test Census data.csv'
current_year = 2021
previous_year = 2016

In [7]:
# Load data
df = pd.read_csv('./data/{}'.format(census_file))

In [8]:
# Review head
df.head()

,Geographic level,Geographic name,Statistic,Value
0,Country,Canada,"Population, 2021",36991981.0
1,Country,Canada,"Population, 2016",35151728.0
2,Country,Canada,0 to 4 years,1831195.0
3,Country,Canada,5 to 9 years,2054900.0
4,Country,Canada,10 to 14 years,2126705.0


# 1. Population Analysis
### Calculate Population Growth and Age Distribution for Canada and each province/territory 

In [ ]:
# Filter age data rows by provided groups.
df_metrics = df[df['Statistic'].str.contains('years')]

# Strip whitespaces from 'Statistic' column
df_metrics['Statistic'] = df_metrics['Statistic'].str.lstrip()

# Map old age groups to new age groups
def map_age(age_str):
    """
    Maps a shorter age group to existing age group category in data.

    Parameters:
        age_str (int or float): The existing age group.

    Returns:
        str: The new age group category as a string.
             Returns 'Unknown' if the age does not fit any category.
    """
    if 'under' in age_str.lower() or '0 to 4' in age_str or '5 to 9' in age_str or '10 to 14' in age_str:
        return '0 to 14 years'
    elif '15 to 19' in age_str:
        return '15 to 19 years'
    elif any(x in age_str for x in ['20 to', '25 to', '30 to', '35 to', '40 to', '45 to', '50 to', '55 to', '60 to', '20 to 64']):
        return '20 to 64 years'
    elif any(x in age_str for x in ['65 to', '70 to', '75 to', '80 to', '85 to', '90 to', '95 to', '100']):
        return '65 years and over'
    else:
        return 'Unknown'

df_metrics['New Statistic'] = df_metrics['Statistic'].apply(map_age)

# Calculate age aggregates based on new mapping
df_metrics = df_metrics.groupby(['Geographic name', 'New Statistic'])['Value'].sum().reset_index()

# Calculate age distributions
def calculate_age_distribution(df):
    """
    Calculates the percentage of each age group within each geographic region.
    
    Parameters:
    df (pd.DataFrame): Input DataFrame with columns ['Geographic name', 'New Statistic', 'Value']
    
    Returns:
    pd.DataFrame: DataFrame with an additional column 'Age Distribution' (%)
    """
    # Calculate the total population per geographic name
    total_per_geo = df.groupby('Geographic name')['Value'].transform('sum')
    
    # Calculate age distribution as percentage
    df['Age Distribution'] = (df['Value'] / total_per_geo) * 100
    
    return df

df_metrics = calculate_age_distribution(df_metrics)

# Pivot dataframe
df_metrics = df_metrics.pivot_table(index='Geographic name', columns='New Statistic', values='Value', aggfunc='sum').reset_index()

# Total population (Current Year)
df_metrics['Total Population ' + str(current_year)] = df_metrics.iloc[:, 1:5].sum(axis=1).rename('Total Population')

# Total population (Previous Year)
df_pop_prev_yr = df.loc[df['Statistic'].str.contains('Population, ' + str(previous_year)), ['Geographic name', 'Value']].rename(columns={'Value': 'Total Population ' + str(previous_year)})
df_metrics = df_metrics.merge(df_pop_prev_yr, on='Geographic name', how='left')

# Population Growth (%)
df_metrics['Population Growth (%)'] = ((df_metrics['Total Population ' + str(current_year)] - df_metrics['Total Population ' + str(previous_year)]) / df_metrics['Total Population ' + str(previous_year)]) * 100

# Median household income
df_med_inc = df.loc[df['Statistic'] == 'Median total income of household in {} ($)'.format(str(current_year - 1)), ['Geographic name', 'Value']].rename(columns={'Value': 'Median total income of household in ' + str(current_year - 1)})
df_metrics = df_metrics.merge(df_med_inc, on='Geographic name', how='left')

# Unemployment Rate
df_unemp_rate = df.loc[df['Statistic'] == 'Unemployment rate', ['Geographic name', 'Value']].rename(columns={'Value': 'Unemployment Rate (%)'})
df_metrics = df_metrics.merge(df_unemp_rate, on='Geographic name', how='left')

# Sort values by population size
df_metrics.sort_values(by='Total Population ' + str(current_year), ascending=False, inplace=True)
df_metrics.reset_index(drop=True, inplace=True)

# Save file
df_metrics.to_csv('./data/population_metrics.csv', index=False)

df_metrics

/var/folders/c7/c3yy380s1x7cgg4pwxmqv46m0000gn/T/ipykernel_20255/1817236903.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_metrics['Statistic'] = df_metrics['Statistic'].str.lstrip()
/var/folders/c7/c3yy380s1x7cgg4pwxmqv46m0000gn/T/ipykernel_20255/1817236903.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_metrics['New Statistic'] = df_metrics['Statistic'].apply(map_age)


,Geographic name,0 to 14 years,15 to 19 years,20 to 64 years,65 years and over,Total Population 2021,Total Population 2016,Population Growth (%),Median total income of household in 2020,Unemployment Rate (%)
0,Canada,8480585.0,2012975.0,19545375.0,6953045.0,36991980.0,35151728.0,5.235168,84000.0,10.3
1,Ontario,3169955.0,801455.0,7642825.0,2609705.0,14223940.0,13448494.0,5.766043,91000.0,12.2
2,Quebec,1979590.0,429400.0,4354975.0,1737865.0,8501830.0,8164361.0,4.133440,72500.0,7.6
3,British Columbia,1046435.0,253690.0,2694190.0,1006570.0,5000885.0,4648055.0,7.590917,85000.0,8.4
4,Alberta,1122790.0,249765.0,2266340.0,623735.0,4262630.0,4067175.0,4.805670,96000.0,11.5
5,Manitoba,340105.0,80905.0,694735.0,226405.0,1342150.0,1278365.0,4.989577,79500.0,8.3
6,Saskatchewan,298610.0,68510.0,569840.0,195550.0,1132510.0,1098352.0,3.109932,82000.0,8.4
7,Nova Scotia,194935.0,48180.0,512755.0,213515.0,969385.0,923598.0,4.957460,71500.0,12.7
8,New Brunswick,160045.0,39090.0,400905.0,175555.0,775595.0,747101.0,3.813942,70000.0,10.3
9,Newfoundland and Labrador,99700.0,25720.0,265130.0,119995.0,510545.0,519716.0,-1.764618,71500.0,15.2


# 2 & 3. Summarize Results & Automate Process
### Populate Population_summary.xlsx file with data for Canada and each province/territory with complete automation

In [ ]:
def copy_sheet_format(source_sheet, target_sheet):
    for row in source_sheet.iter_rows():
        for cell in row:
            new_cell = target_sheet.cell(row=cell.row, column=cell.column, value=cell.value)

            if cell.has_style:
                new_cell.font = copy(cell.font)
                new_cell.border = copy(cell.border)
                new_cell.fill = copy(cell.fill)
                new_cell.number_format = copy(cell.number_format)
                new_cell.protection = copy(cell.protection)
                new_cell.alignment = copy(cell.alignment)

    # Copy column widths
    for col in source_sheet.column_dimensions:
        target_sheet.column_dimensions[col].width = source_sheet.column_dimensions[col].width

    # Copy row heights
    for row in source_sheet.row_dimensions:
        target_sheet.row_dimensions[row].height = source_sheet.row_dimensions[row].height

def create_metrics_excel(file_path, sheet_names):
    wb = load_workbook(file_path)
    original_sheet = wb.active

    for name in sheet_names:
        new_sheet = wb.copy_worksheet(original_sheet)
        new_sheet.title = name
        copy_sheet_format(original_sheet, new_sheet)
        new_sheet['B2'] = name
        new_sheet['C4'] = df_metrics.loc[df_metrics['Geographic name'] == name]['Total Population ' + str(current_year)].values[0]
        new_sheet['C5'] = df_metrics.loc[df_metrics['Geographic name'] == name]['Population Growth (%)'].values[0]
        new_sheet['C8'] = df_metrics.loc[df_metrics['Geographic name'] == name]['0 to 14 years'].values[0]
        new_sheet['C9'] = df_metrics.loc[df_metrics['Geographic name'] == name]['15 to 19 years'].values[0]
        new_sheet['C10'] = df_metrics.loc[df_metrics['Geographic name'] == name]['20 to 64 years'].values[0]
        new_sheet['C11'] = df_metrics.loc[df_metrics['Geographic name'] == name]['65 years and over'].values[0]
        new_sheet['C13'] = df_metrics.loc[df_metrics['Geographic name'] == name]['Median total income of household in ' + str(current_year - 1)].values[0]
        new_sheet['C15'] = df_metrics.loc[df_metrics['Geographic name'] == name]['Unemployment Rate (%)'].values[0]

        # Disable gridlines view
        new_sheet.sheet_view.showGridLines = False

    # Remove the original sheet
    wb.remove(original_sheet)

    # Rename the first sheet to 'Canada'
    wb.worksheets[0].title = 'Canada'

    wb.save(file_path)
    wb.close()

# Create list of regions
region_list = df_metrics['Geographic name'].tolist()

# Define Excel Summary file name
file_name = './data/{}'.format(pop_summary_file)

# Create summary workshsheets
create_metrics_excel(file_name, region_list)

# 4. Innvoation
### Created a streamlit application that takes in the summarised CSV file as input (data/population_metrics.csv), and loads the data in a web browser
### in the command line, install the requirements file (pip install -r requirements.txt) and run the Streamlit application (run streamlit run app.py)


In [ ]:
# 5. Further Improvement Recommendations
#### a. There is a minute difference in the population numbers when comparing total population number with age-wise breakdown, however, it is negligible and can be attributed to rounding errors or data discrepancies.
#### b. The code can be further optimized by using vectorized operations instead of applying functions row-wise, which can improve performance on larger datasets.
#### c. Additional error handling can be implemented to manage cases where the expected data might not be present in the input files.
# 6. Conclusion
# The code successfully processes the census data to generate a population metrics summary, including age distributions, total population, growth rates, median income, and unemployment rates for various geographic regions. The results are saved in a structured format suitable for further analysis or reporting.
# 7. References
# - Pandas documentation: https://pandas.pydata.org/docs/
# - Openpyxl documentation: https://openpyxl.readthedocs.io/en/stable/
# - Python documentation: https://docs.python.org/3/
# 8. License
# This code is released under the MIT License. You are free to use, modify, and distribute it as per the terms of the license.
# 9. Contact Information
# For any questions or issues regarding this code, please contact the author at [your email address].
# 4. Code Explanation



# This code processes census data to generate a population metrics summary for various geographic regions. It includes age distributions, total population, growth rates, median income, and unemployment rates. The results are saved in a structured format suitable for further analysis or reporting.
# 1. Introduction
# This code is designed to process census data and generate a population metrics summary for various geographic regions. It includes calculations for age distributions, total population, growth rates, median income, and unemployment rates. The results are saved in a structured format suitable for further analysis or reporting.
# 2. Requirements






# - Python 3.x
# - Pandas library



















# - Openpyxl library









